In [1]:
import requests
import json
response_API1 = requests.get('https://covid19.ddc.moph.go.th/api/Vaccinated/weekly-vaccinated-by-provice')
response_API2 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/today-cases-by-provinces')
response_API3 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/today-cases-line-lists')
response_API4 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/timeline-cases-by-provinces')
overall_API5 = requests.get('https://covid19.ddc.moph.go.th/api/Cases/timeline-cases-all')
#print(response_API.status_code)

In [115]:
import sqlite3

conn = sqlite3.connect('Covid.db')
cursor = conn.cursor()
#cursor.execute('PRAGMA table_info(Covid)').fetchall()

# cursor.execute('''CREATE TABLE vaccine
#                 (year int, weeknum int, province text, vaccine_total int, vaccine_1st_dose int, vaccine_2nd_dose int, vaccine_3rd_dose int, vaccine_total_acm int, vaccine_1st_dose_acm int, vaccine_2nd_dose_acm int, vaccine_3rd_dose_acm int, region text)''')

# cursor.execute('''CREATE TABLE covid
#                 (year int, weeknum int, province text, new_case int, total_case int, new_case_excludeabroad int, total_case_excludeabroad int, new_death int, total_death int, update_date text, region text)''')

# conn.commit()

In [116]:
province_north = ["เชียงราย","เชียงใหม่","น่าน","พะเยา","แพร่","แม่ฮ่องสอน","ลำปาง","ลำพูน","อุตรดิตถ์"]
province_south = ["กระบี่","ชุมพร","ตรัง","นครศรีธรรมราช","นราธิวาส","ปัตตานี","พังงา","พัทลุง","ภูเก็ต","ยะลา","ระนอง","สงขลา","สตูล","สุราษฎร์ธานี"]
province_central = ["กรุงเทพมหานคร","กำแพงเพชร","ชัยนาท","นครนายก","นครปฐม","นครสวรรค์","นนทบุรี","ปทุมธานี","พระนครศรีอยุธยา","พิจิตร","พิษณุโลก","เพชรบูรณ์","ลพบุรี","สมุทรปราการ","สมุทรสงคราม","สมุทรสาคร","สระบุรี","สิงห์บุรี","สุโขทัย","สุพรรณบุรี","อ่างทอง","อุทัยธานี"]
province_west = ["กาญจนบุรี","ตาก","ประจวบคีรีขันธ์","เพชรบุรี","ราชบุรี"]
province_east = ["จันทบุรี","ฉะเชิงเทรา","ชลบุรี","ตราด","ปราจีนบุรี","ระยอง","สระแก้ว"]
province_northeast = ["กาฬสินธุ์","ขอนแก่น","ชัยภูมิ","นครพนม","นครราชสีมา","บึงกาฬ","บุรีรัมย์","มหาสารคาม","มุกดาหาร","ยโสธร","ร้อยเอ็ด","เลย","ศรีสะเกษ","สกลนคร","สุรินทร์","หนองคาย","หนองบัวลำภู","อำนาจเจริญ","อุดรธานี","อุบลราชธานี"]

In [117]:
#ดึงข้อมูลจาก api ปี2021 - ปัจจุบัน
allcase = response_API4.text

allcase = json.loads(allcase)
for i in allcase:
    if i["province"] == "ทั้งประเทศ":
        i["region"] = 'ทั้งประเทศ'
    elif i["province"] == "ไม่ระบุ":
        i["region"] = 'ไม่ระบุ'
    elif i["province"] in province_north:
        i["region"] = 'ภาคเหนือ'
    elif i["province"] in province_south:
        i["region"] = 'ภาคใต้'
    elif i["province"] in province_central:
        i["region"] = 'ภาคกลาง'
    elif i["province"] in province_west:
        i["region"] = 'ภาคตะวันตก'
    elif i["province"] in province_east:
        i["region"] = 'ภาคตะวันออก'
    elif i["province"] in province_northeast:
        i["region"] = 'ภาคตะวันออกเฉียงเหนือ'
    data = (i['update_date'], i['province'], i['region'], i['year'], i['weeknum'], i['new_case'], i['total_case'], i['new_case_excludeabroad'], i['total_case_excludeabroad'], i['new_death'], i['total_death'])
      
    # conn.execute("INSERT INTO covid (update_date ,province ,region ,year ,weeknum, new_case ,total_case ,new_case_excludeabroad ,total_case_excludeabroad ,new_death ,total_death ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",data)
    # conn.commit()

In [118]:
import pandas as pd
from tabulate import tabulate

vaccinated_by_provice = response_API1.text
today_cases_by_province = response_API2.text
died_cases = response_API3.text

data1 = json.loads(vaccinated_by_provice)
data2 = json.loads(today_cases_by_province)


for i in data1:
    if i["province"] == "ทั้งประเทศ":
        i["region"] = 'ทั้งประเทศ'
    elif i["province"] == "ไม่ระบุ":
        i["region"] = 'ไม่ระบุ'
    for j in province_north:
        if i["province"] == j:
            i["region"] = 'ภาคเหนือ'
    for j in province_south:
        if i["province"] == j:
            i["region"] = 'ภาคใต้'
    for j in province_central:
        if i["province"] == j:
            i["region"] = 'ภาคกลาง'
    for j in province_west:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันตก'
    for j in province_east:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออก'
    for j in province_northeast:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออกเฉียงเหนือ'
    vaccine_data = tuple(i.values())
    # conn.execute("INSERT INTO vaccine VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", vaccine_data)
  


In [119]:
for i in data2:
    if i["province"] == "ทั้งประเทศ":
        i["region"] = 'ทั้งประเทศ'
    elif i["province"] == "ไม่ระบุ":
        i["region"] = 'ไม่ระบุ'
    for j in province_north:
        if i["province"] == j:
            i["region"] = 'ภาคเหนือ'
    for j in province_south:
        if i["province"] == j:
            i["region"] = 'ภาคใต้'
    for j in province_central:
        if i["province"] == j:
            i["region"] = 'ภาคกลาง'
    for j in province_west:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันตก'
    for j in province_east:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออก'
    for j in province_northeast:
        if i["province"] == j:
            i["region"] = 'ภาคตะวันออกเฉียงเหนือ'
    covid_data = tuple(i.values())
    # cursor.execute("INSERT INTO covid VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", covid_data)

In [120]:
# cursor.execute('''CREATE TABLE alldata AS SELECT * FROM covid LEFT OUTER JOIN vaccine ON covid.province = vaccine.province;''')
# cursor.execute('''alter table alldata drop 'year:1' ''')
# cursor.execute('''alter table alldata drop 'weeknum:1' ''')
# cursor.execute('''alter table alldata drop 'province:1' ''')
# cursor.execute('''alter table alldata drop 'region:1' ''')

In [121]:
##เพิ่มข้อมูลเข้า table alldata
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('covid.db')

# Get a cursor object
cursor = conn.cursor()

# Execute the SELECT statement
cursor.execute("SELECT * FROM covid WHERE year != 2023")

# Fetch all the rows returned by the query
rows = cursor.fetchall()

# Insert the selected rows into the "alldata" table
for i in rows:
    data = (i +(0,0,0,0,0,0,0,0))
    # cursor.execute("INSERT INTO alldata VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?,?,?,?,?,?,?)", data)

# Commit the changes
# conn.commit()

# Close the cursor and connection objects
# cursor.close()
# conn.close()


In [122]:
year 2023
weeknum 9
province ทั้งประเทศ
vaccine_total 727
vaccine_1st_dose 73
vaccine_2nd_dose 153
vaccine_3rd_dose 501
vaccine_total_acm 144872431
vaccine_1st_dose_acm 57226251
vaccine_2nd_dose_acm 53719574
vaccine_3rd_dose_acm 33926606

SyntaxError: invalid syntax (2646373387.py, line 1)

In [ ]:
import urllib
url = 'https://opend.data.go.th/get-ckan/datastore_search?resource_id=0c2aa4d6-90ef-405f-acf2-20507ced4fab&limit=5&q=title:jones'  
fileobj = json.loads(requests.get(url).text)
print (fileobj)

{'message': 'No API key found in request'}


In [ ]:
import json
import sqlite3

# Open the JSON file and load its contents
with open('Thailandprovinces.json', 'r') as f:
    province = json.load(f)
# Extract the new value for the "province" column
# Thailandprovinces

# # Connect to the SQLite database
conn = sqlite3.connect('covid.db')

# # Get a cursor object
cursor = conn.cursor()

# # Execute the UPDATE statement
# for i in data.keys():
#     new_province = data.get(i)
#     #print(i,new_province)
#     conn.execute("UPDATE alldata SET province = ? WHERE province = ?", (new_province, i))
#     conn.commit()
    
# conn.execute("UPDATE alldata set province = ? WHERE province = ?", ('None', 'ไม่ระบุ'))
# conn.commit()
# conn.execute("UPDATE alldata set province = ? WHERE province = ?", ('All', 'ทั้งประเทศ'))
# conn.commit()
# # Commit the changes



In [ ]:
region = {"ทั้งประเทศ":"All","ไม่ระบุ":'None','ภาคเหนือ':'North','ภาคใต้':'South','ภาคกลาง':'Central','ภาคตะวันตก':'West','ภาคตะวันออก':'East','ภาคตะวันออกเฉียงเหนือ':'NorthEast'}
for i in region.keys():
    new_region = region.get(i)
    # conn.execute("UPDATE alldata_province_eng SET region = ? WHERE region = ?", (i, new_region))
    # conn.commit()

In [131]:
for i in province.keys():
    new_province = province.get(i)
#     conn.execute("UPDATE alldata_province_th SET province = ? WHERE province = ?", (i, new_province))
#     conn.commit()

# conn.execute("UPDATE alldata_province_th SET province = ? WHERE province = ?", ("ทั้งประเทศ", "All"))
# conn.commit()
# conn.execute("UPDATE alldata_province_th SET province = ? WHERE province = ?", ("ไม่ระบุ", "None"))
# conn.commit()



In [7]:
overall = overall_API5.text

overall = json.loads(overall)
print(overall)

[{'year': 2021, 'weeknum': 52, 'new_case': 3011, 'total_case': 3011, 'new_case_excludeabroad': 2869, 'total_case_excludeabroad': 2869, 'new_recovered': 3315, 'total_recovered': 3315, 'new_death': 10, 'total_death': 10, 'case_foreign': 142, 'case_prison': 57, 'case_walkin': 2716, 'case_new_prev': 3111, 'case_new_diff': -100, 'death_new_prev': 26, 'death_new_diff': -16, 'update_date': '2023-03-17 19:32:23'}, {'year': 2022, 'weeknum': 1, 'new_case': 34593, 'total_case': 37604, 'new_case_excludeabroad': 33067, 'total_case_excludeabroad': 35936, 'new_recovered': 19453, 'total_recovered': 22768, 'new_death': 105, 'total_death': 115, 'case_foreign': 1526, 'case_prison': 319, 'case_walkin': 31802, 'case_new_prev': 29341, 'case_new_diff': 5252, 'death_new_prev': 101, 'death_new_diff': 4, 'update_date': '2023-03-17 19:32:23'}, {'year': 2022, 'weeknum': 2, 'new_case': 55369, 'total_case': 92973, 'new_case_excludeabroad': 53168, 'total_case_excludeabroad': 89104, 'new_recovered': 25862, 'total_rec

In [9]:
import sqlite3

# Create a connection to the database
conn = sqlite3.connect('covid.db')

# Create a cursor object
cursor = conn.cursor()

# Create the table
cursor.execute('''CREATE TABLE overall_data (
                    year INTEGER,
                    weeknum INTEGER,
                    new_case INTEGER,
                    total_case INTEGER,
                    new_case_excludeabroad INTEGER,
                    total_case_excludeabroad INTEGER,
                    new_recovered INTEGER,
                    total_recovered INTEGER,
                    new_death INTEGER,
                    total_death INTEGER,
                    case_foreign INTEGER,
                    case_prison INTEGER,
                    case_walkin INTEGER,
                    case_new_prev INTEGER,
                    case_new_diff INTEGER,
                    death_new_prev INTEGER,
                    death_new_diff INTEGER,
                    update_date TEXT)''')

In [10]:
for row in overall:
    cursor.execute('''INSERT INTO overall_data (
                        year,
                        weeknum,
                        new_case,
                        total_case,
                        new_case_excludeabroad,
                        total_case_excludeabroad,
                        new_recovered,
                        total_recovered,
                        new_death,
                        total_death,
                        case_foreign,
                        case_prison,
                        case_walkin,
                        case_new_prev,
                        case_new_diff,
                        death_new_prev,
                        death_new_diff,
                        update_date) 
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                    (row['year'], row['weeknum'], row['new_case'], row['total_case'], row['new_case_excludeabroad'], row['total_case_excludeabroad'], row['new_recovered'], row['total_recovered'], row['new_death'], row['total_death'], row['case_foreign'], row['case_prison'], row['case_walkin'], row['case_new_prev'], row['case_new_diff'], row['death_new_prev'], row['death_new_diff'], row['update_date']))

# Commit the changes and clalose the connection
conn.commit()
conn.close()

In [13]:
conn = sqlite3.connect('covid.db')
conn.execute('''select  total_case,total_death,total_recovered  from overall_data where year = 2022 and weeknum=52''').fetchall()

[(2500484, 11971, 2478781)]